In [1]:
import requests
import lxml.html
import bs4

import numpy as np
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
LOGIN_URL = 'https://www.placement.iitbhu.ac.in/accounts/login'
COMPANY_URL = 'https://www.placement.iitbhu.ac.in/company/calendar'
USERNAME = 'sartaj.singh.apm13@itbhu.ac.in'
PASSWORD = 'Superkid@123'

In [3]:
def make_login_form(session, username, password):
    login = session.get(LOGIN_URL)
    login_html = lxml.html.fromstring(login.text)
    hidden_inputs = login_html.xpath(r'//form//input[@type="hidden"]')
    form = {x.attrib['name']: x.attrib['value'] for x in hidden_inputs}
    form['login'] = username
    form['password'] = password
    return form

In [4]:
session = requests.session()

In [5]:
form = make_login_form(session, USERNAME, PASSWORD)
form

{'csrfmiddlewaretoken': '5HYQ3od5urJ0nSuA0iEZP8lNRoCuYJRI',
 'login': 'sartaj.singh.apm13@itbhu.ac.in',
 'password': 'Superkid@123'}

In [6]:
params = {'Referer': LOGIN_URL, 'User-Agent': USER_AGENT}
login = session.post(LOGIN_URL, data=form, headers=params)
login

<Response [200]>

In [7]:
params = {'page_size': 250}
data = session.get(COMPANY_URL, params=params)
data

<Response [200]>

In [8]:
company_data = bs4.BeautifulSoup(data.text, 'html.parser')

In [9]:
data_mapper = {
    'updated_at': 'updated_at',
    'company_name': 'company_name',
    'profile': 'company_profile',
    'purpose': 'purpose',
    'x_percent': 'x',
    'xii_percent': 'xii',
    'cgpa': 'cgpa',
    'course': 'course',
    'dept': 'dept',
    'active_backlog': 'a_backlog',
    'total_backlog': 't_backlog',
    'ppt_date': 'ppt_date',
    'exam_date': 'exam_date',
    'interview_date': 'interview_date',
    'status': 'status',
    'branch_issue_deadline': 'branch_issue_dead',
    'willingness_deadline': 'willingness_dead',
}

In [10]:
clips = {
    'active_backlog': 'Active backlog allowed :'.find(':'),
    'active_backlog2': 'Active backlog allowed :'.find(':') ,
    'active_backlog3': 'Active backlog allowed :'.find(':') ,
    'branch_issue_deadline': 'Branch Issues Deadline :'.find(':') + 1,
    'cgpa': 'cgpa :'.find(':'),
    'cgpa1': 'cgpa :'.find(':'),
    'cgpa2': 'cgpa :'.find(':'),
    'course': 'Course(s) :'.find(':'),
    'course1': 'Course(s) :'.find(':'),
    'course2': 'Course(s) :'.find(':'),
    'dept': 'Department(s) :'.find(':'),
    'dept1': 'Department(s) :'.find(':'),
    'dept2': 'Department(s) :'.find(':'),
    'exam_date': 'Exam Date :'.find(':'),
    'interview_date': 'Interview Date :'.find(':') ,
    'ppt_date': 'PPT Date : '.find(':'),
    'status': 'Status : '.find(':'),
    'total_backlog': 'Total backlog allowed :'.find(':'),  
    'updated_at': 'st Updated:'.find(':') + 2,
    'willingness_deadline': 'Willingness Deadline :'.find(':') + 1,
    'x_percent': 'X :'.find(':'),
    'x_percent1': 'X :'.find(':'),
    'x_percent2': 'X :'.find(':'),
    'xii_percent': 'XII :'.find(':'),
    'xii_percent1': 'XII :'.find(':'),
    'xii_percent2': 'XII :'.find(':')
}

In [11]:
def get_package_data(company_div, company):
    package = company_div.find('div', attrs={'class': 'package'})
    for tr in package.find_all('tr'):
        course, num = False, 0
        course_name = None
        for td in tr.find_all('td'):
            if course:
                if num == 0 and td.text:
                    company['{}_ctc'.format(course_name)] = td.text
                if num == 1 and td.text:
                    company['{}_home'.format(course_name)] = td.text
                if num == 1:
                    course = False
                num += 1
            if td.text.lower() in ['b.tech', 'idd/imd', 'm.tech', 'phd']:
                course, num = True, 0
                course_name = td.text.lower().replace('/', '_').replace('.', '')

In [12]:
def generate_names(name):
    i = 0
    while True:
        if i:
            yield '{}{}'.format(name, i)
        else:
            yield name
        i += 1

In [13]:
def get_data(data_mapper):
    companies = []
    for company_div in company_data.find_all('div', attrs={'class': 'row company'}):
        company = {}
        for name, class_name in data_mapper.items():
            names = generate_names(name)
            for name, tag in zip(names, company_div.find_all(attrs={'class': class_name})):
                company[name] = tag.text
        get_package_data(company_div, company)
        companies.append(company)
    return companies

In [14]:
def clip_data(companies, clips):
    for company in companies:
        for clip_name, clip_num in clips.items():
            num = clip_num + 2
            if clip_name in company.keys():
                company[clip_name] = company[clip_name][num:]
    return companies

In [15]:
def find_depts(df):
    depts = set([])
    for value in df.dept.unique():
        for dept in value.split(' '):
            depts.add(dept)
    return depts

In [16]:
def _average(values):
    ans, num = 0, 0
    for val in values:
        try:
            ans += float(val)
            num += 1
        except ValueError:
            continue
    if num:
        return ans / num

In [17]:
def package_value(value):
    if not isinstance(value, float):
        ans = _average(value.split(' '))
        if ans:
            if ans < 100:
                ans *= 10**5
            return ans
        
    return value

In [18]:
def package_value_series(packages):
    return packages.apply(package_value)

In [19]:
def process_packages(df):
    packages = []
    courses = ['btech', 'idd_imd', 'mtech', 'phd']
    packages.extend(map(lambda x: '{}_ctc'.format(x), courses))
    packages.extend(map(lambda x: '{}_home'.format(x), courses))

    df.loc[:, packages] = df[packages].replace('Yet to Decide', np.nan)
    df.loc[:, packages] = df[packages].replace('[lL]akhs', '', regex=True)
    df.loc[:, packages] = df[packages].replace('LPA', '', regex=True)
    df.loc[:, packages] = df[packages].replace('\+ plus other benefits', '', regex=True)
    df.loc[:, packages] = df[packages].replace(',', '', regex=True)
    df.loc[:, packages] = df[packages].replace('[-/]', ' ', regex=True)
    df.loc[:, packages] = df[packages].replace('Depending on experience', np.nan, regex=True)
    df.loc[:, packages] = df[packages].replace('Inclusive of 20% variable pay', np.nan)
    
    df.loc[:, packages] = df[packages].apply(package_value_series)
    df.loc[df.company_name == 'VMWare', ['idd_imd_ctc', 'btech_ctc']] = [2202108, 2202108]
    df.loc[:, packages] = df[packages].astype(np.float64)
          
    return df

In [20]:
def process_df(companies):
    df = pd.DataFrame(companies)
    
    df.loc[:, 'active_backlog'] = df['active_backlog'].replace({'N/A': 0}).astype(np.float64)
    df.loc[:, 'total_backlog'] = df['total_backlog'].replace({'N/A': 0}).astype(np.float64)
    
    df.loc[:, 'cgpa'] = df[['cgpa', 'cgpa1', 'cgpa2']].astype(np.float64).mean(axis=1)
    df.loc[:, 'exam_date'] = pd.to_datetime(df['exam_date'])
    df.loc[:, 'interview_date'] = pd.to_datetime(df['interview_date'])
    df.loc[:, 'ppt_date'] = pd.to_datetime(df['ppt_date'])
    df.loc[:, 'branch_issue_deadline'] = pd.to_datetime(df['branch_issue_deadline'].str.replace('|', ','))
    df.loc[:, 'updated_at'] = pd.to_datetime(df['updated_at'])
    df.loc[:, 'x_percent'] = df[['x_percent', 'x_percent1', 'x_percent2']].apply(lambda x: x.str[:-1]).astype(np.float64).mean(axis=1)
    df.loc[:, 'xii_percent'] = df[['xii_percent', 'xii_percent1', 'xii_percent2']].apply(lambda x: x.str[:-1]).astype(np.float64).mean(axis=1)
    df.loc[:, 'profile'] = df['profile'].replace({'': np.nan})
    df.loc[:, 'willingness_deadline'] = pd.to_datetime(df['willingness_deadline'].str.replace('|', ','))
    
    df.loc[:, 'dept'] = df['dept1'].fillna('') + ' ' + df['dept2'].fillna('') + df['dept'].fillna('')
    for dept in find_depts(df):
        df.loc[:, dept] = df.dept.apply(lambda x: 1 if dept in x else 0)
    
    df.loc[:, 'course'] = df[['course', 'course1', 'course2']].fillna('').sum(axis=1)
    for course in ['btech', 'idd', 'mtech', 'phd']:
        df.loc[:, course] = df.course.apply(lambda x: 1 if course in x else 0)
    
    df = process_packages(df)
    df = df.drop(['course', 'purpose', 'active_backlog1', 'active_backlog2',
                 'total_backlog1', 'total_backlog2', 'cgpa1', 'cgpa2',
                 'course1', 'course2', 'dept', 'dept1', 'dept2',
                 'x_percent1', 'x_percent2', 'xii_percent1', 'xii_percent2'], axis=1)
    
    return df

In [21]:
companies = get_data(data_mapper)
clip_data(companies, clips)
df = process_df(companies)
df.head()

,active_backlog,branch_issue_deadline,btech_ctc,btech_home,cgpa,company_name,exam_date,idd_imd_ctc,idd_imd_home,interview_date,...,civ,bme,cse,ece,mst,phe,btech,idd,mtech,phd
0,0.0,2017-09-22 10:00:00,700000.0,NaN,6.0,Honeywell,NaT,NaN,NaN,NaT,...,0,0,0,1,0,0,1,0,1,0
1,0.0,2017-09-22 10:00:00,845000.0,NaN,7.0,SCA Tech,NaT,845000.0,NaN,NaT,...,1,0,1,1,1,0,1,1,1,0
2,0.0,2017-09-22 10:00:00,NaN,NaN,0.0,Evalueserve,NaT,NaN,NaN,NaT,...,0,1,0,0,0,0,0,0,1,0
3,0.0,2017-09-19 10:00:00,2384000.0,NaN,8.0,FLIPKART,2017-10-03,2384000.0,NaN,NaT,...,0,0,0,1,0,0,1,1,0,0
4,0.0,2017-08-08 12:00:00,2384000.0,NaN,0.0,FLIPKART,2017-10-03,2384000.0,NaN,NaT,...,0,0,1,0,0,0,1,1,0,0


In [22]:
df.to_csv('companies.csv', index=False)
pd.read_csv('companies.csv', parse_dates=['branch_issue_deadline', 'exam_date', 'interview_date', 'ppt_date', 'updated_at', 'willingness_deadline']).head()

,active_backlog,branch_issue_deadline,btech_ctc,btech_home,cgpa,company_name,exam_date,idd_imd_ctc,idd_imd_home,interview_date,...,civ,bme,cse,ece,mst,phe,btech,idd,mtech,phd
0,0.0,2017-09-22 10:00:00,700000.0,NaN,6.0,Honeywell,NaT,NaN,NaN,NaT,...,0,0,0,1,0,0,1,0,1,0
1,0.0,2017-09-22 10:00:00,845000.0,NaN,7.0,SCA Tech,NaT,845000.0,NaN,NaT,...,1,0,1,1,1,0,1,1,1,0
2,0.0,2017-09-22 10:00:00,NaN,NaN,0.0,Evalueserve,NaT,NaN,NaN,NaT,...,0,1,0,0,0,0,0,0,1,0
3,0.0,2017-09-19 10:00:00,2384000.0,NaN,8.0,FLIPKART,2017-10-03,2384000.0,NaN,NaT,...,0,0,0,1,0,0,1,1,0,0
4,0.0,2017-08-08 12:00:00,2384000.0,NaN,0.0,FLIPKART,2017-10-03,2384000.0,NaN,NaT,...,0,0,1,0,0,0,1,1,0,0
